In [1]:
#!/usr/bin/env python3

# Import libraries
import numpy as np
import pandas as pd
import re
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_multiple_whitespaces

DATA_DIR = 'data/'

train_file = DATA_DIR + 'train.csv'
test_file = DATA_DIR + 'test.csv'


def convert_to_numpy(df):
    print(df)


def load_test_data():
    return pd.read_csv(test_file, header=0)

def load_train_data():
    df = pd.read_csv(train_file, header=0)
    return df.iloc[:,:-1], df.iloc[:,-1]


if __name__ == "__main__":
    dataX, datay = load_train_data()
    print(dataX.head())
    print(datay.head())


   id keyword location                                               text
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...
0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64


In [2]:
# Compile url regex (a basic one)
url_regex = re.compile("(http|https)://[A-Za-z0-9_-]+\.[A-Za-z0-9_-]+(/\S*)?")
# Compile regex to detect tokens that are entirely non-text
nontext_regex = re.compile("[^A-Za-z]+")
# Compile regex to detect @ mentions
mention_regex = re.compile("@\S+")
# Compile regex to detect various mis-encoded punctuation characters
misenc_regex = re.compile("&amp;")
# Compile regex to check if text is composed entirely of letters and digits
alphanum_regex = re.compile("[A-Za-z0-9]+")


In [3]:
dataX['text']

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
5       #RockyFire Update => California Hwy. 20 closed...
6       #flood #disaster Heavy rain causes flash flood...
7       I'm on top of the hill and I can see a fire in...
8       There's an emergency evacuation happening now ...
9       I'm afraid that the tornado is coming to our a...
10            Three people died from the heat wave so far
11      Haha South Tampa is getting flooded hah- WAIT ...
12      #raining #flooding #Florida #TampaBay #Tampa 1...
13                #Flood in Bago Myanmar #We arrived Bago
14      Damage to school bus on 80 in multi car crash ...
15                                         What's up man?
16                                          I love fruits
17            

In [4]:
text = dataX['text'][0]
print(text)
text = url_regex.sub("", text)
text = misenc_regex.sub("", text)
text = mention_regex.sub("", text)
text = re.sub("#", "", text)
text = remove_stopwords(text)
text = strip_numeric(text)
text = strip_punctuation(text)
text = strip_multiple_whitespaces(text)
words = text.split(" ")
print(words)

'''
for i in dataX['text']:
    text = url_regex.sub("", text)
    text = misenc_regex.sub("", text)
    text = mention_regex.sub("", text)
    text = re.sub("#", "", text)
    text = remove_stopwords(text)
    words = text.split(" ")
    
print(words)    
'''

Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
['Our', 'Deeds', 'Reason', 'earthquake', 'May', 'ALLAH', 'Forgive']


'\nfor i in dataX[\'text\']:\n    text = url_regex.sub("", text)\n    text = misenc_regex.sub("", text)\n    text = mention_regex.sub("", text)\n    text = re.sub("#", "", text)\n    text = remove_stopwords(text)\n    words = text.split(" ")\n    \nprint(words)    \n'

In [5]:
sentence = []
#for i in dataX['text'][0:10]:
for i in dataX['text']:
    text = url_regex.sub("", i)
    text = misenc_regex.sub("", text)
    text = mention_regex.sub("", text)
    text = re.sub("#", "", text)
    text = remove_stopwords(text)
    text = strip_numeric(text)
    text = strip_punctuation(text)
    text = strip_multiple_whitespaces(text)
    words = text.split(" ")
    sentence.append(words)
#print(sentence) 

In [6]:
from gensim.models.fasttext import FastText

# use example set of texts
from gensim.test.utils import common_texts

#build the model and train
#use vector size 4 for the example
model = FastText(size=4, window=3, min_count=1)  # instantiate
model.build_vocab(sentences=common_texts)
model.train(sentences=common_texts, total_examples=len(common_texts), epochs=10)  # train

model.wv['computer']

#produce a list of word embeddings for the example
veclist = []
for i in common_texts[0:len(common_texts)]:
    veclist.append(model.wv[i])
print(veclist)
    

[array([[ 0.01575068,  0.05351213, -0.01825128,  0.01451734],
       [ 0.0185172 ,  0.03048961, -0.00839526,  0.02434803],
       [-0.01146877,  0.05369632, -0.04157733, -0.0052736 ]],
      dtype=float32), array([[-0.03524481,  0.03532043,  0.04959469, -0.02352963],
       [-0.03324692, -0.06451701, -0.01015481, -0.04317482],
       [-0.01146877,  0.05369632, -0.04157733, -0.0052736 ],
       [-0.03050808, -0.00921003,  0.0233555 ,  0.00851412],
       [ 0.02457302, -0.05300654, -0.00205131,  0.00771587],
       [-0.01400802,  0.04563731,  0.0210558 ,  0.02731517]],
      dtype=float32), array([[ 0.13908893,  0.0225455 ,  0.02700579,  0.01246016],
       [-0.03324692, -0.06451701, -0.01015481, -0.04317482],
       [ 0.0185172 ,  0.03048961, -0.00839526,  0.02434803],
       [-0.03050808, -0.00921003,  0.0233555 ,  0.00851412]],
      dtype=float32), array([[-0.03050808, -0.00921003,  0.0233555 ,  0.00851412],
       [ 0.01575068,  0.05351213, -0.01825128,  0.01451734],
       [-0.0305

In [7]:
from gensim.models.fasttext import FastText

#build the model and train
#use vector size 4 for the example
model1 = FastText(size=4, window=3, min_count=1)  # instantiate
#model1.build_vocab(sentences=words)
#model1.train(sentences=words, total_examples=1, epochs=10)  # train
model1.build_vocab(sentences=sentence)
model1.train(sentences=sentence, total_examples=len(sentence), epochs=10)  # train


In [8]:
model1.wv['Deeds']

array([-0.705604  ,  2.8683395 , -0.6167746 , -0.48912165], dtype=float32)

In [9]:
#produce a list of word embeddings for the example
veclist = []
for i in sentence[0:len(sentence)]:
    veclist.append(model.wv[i])
#print(veclist)
    

In [10]:
model1.wv[sentence[0]]
model1.wv[sentence[1]]

array([[-1.502542  ,  5.522774  , -0.88677776, -1.04765   ],
       [-1.1531698 ,  6.3356037 , -0.98533607, -0.9949615 ],
       [-0.8509851 ,  3.3678527 , -0.64768577, -0.5613398 ],
       [-0.76767725,  2.8297114 , -0.43349996, -0.4145956 ],
       [-0.31355256,  1.6160703 , -0.32882726, -0.23860975],
       [-0.57533723,  2.112945  , -0.36981273, -0.30562758]],
      dtype=float32)

In [11]:
print(len(veclist))

7613
